In [12]:
import os
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [13]:
os.chdir(r"C:\Users\SGU\Projects\mlops-with-mlflow")
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [14]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str
    

In [19]:
import os
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib

from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str
    
class Trainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        train_y =train_data[[self.config.target_column]]
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        model.fit(train_x, train_y)
        
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        

In [27]:

from mlProject.utils.constants import CONFIG_FILE_PATH, SCHEMA_FILE_PATH, PARAMS_FILE_PATH
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
        print(self.config)
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.target_columns
        
        create_directories([config.root_dir])
        return ModelTrainerConfig(
            config.root_dir,
            config.train_data_path,
            config.test_data_path,
            config.model_name,
            params.alpha,
            params.l1_ratio,
            schema.name
        )

try:
    cfg_mgr = ConfigurationManager()
    model_trainer_config = cfg_mgr.get_model_trainer_config()
    model_trainer = Trainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-08-12 09:53:41,254: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-12 09:53:41,258: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-12 09:53:41,261: INFO: common: yaml file: params.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_path': 'artifacts/data_ingestion/winequality-red.csv', 'status_file': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'art